In [1]:
import pandas as pd
import scipy as sc
import numpy as np
from lmfit import minimize,Parameters,fit_report
import math

In [2]:
df = pd.read_csv("../data/start_val_data.csv") #read csv data

In [3]:
df

,Unnamed: 0,ID,Time,PopBio,Species,Citation,Rmax,Tlag,Nmax,Tmax,N0,A,H0
0,0,1,669.879518,0.283276,Chryseobacterium.balustinum,"Bae, Y.M., Zheng, L., Hyun, J.E., Jung, K.S., ...",0.000844,43.037977,0.285151,1,0.008629,3.497856,27.046819
1,1,1,646.987952,0.283342,Chryseobacterium.balustinum,"Bae, Y.M., Zheng, L., Hyun, J.E., Jung, K.S., ...",0.000844,43.037977,0.285151,1,0.008629,3.497856,27.046819
2,2,1,622.891566,0.285151,Chryseobacterium.balustinum,"Bae, Y.M., Zheng, L., Hyun, J.E., Jung, K.S., ...",0.000844,43.037977,0.285151,1,0.008629,3.497856,27.046819
3,3,1,597.590361,0.281746,Chryseobacterium.balustinum,"Bae, Y.M., Zheng, L., Hyun, J.E., Jung, K.S., ...",0.000844,43.037977,0.285151,1,0.008629,3.497856,27.046819
4,4,1,574.698795,0.273117,Chryseobacterium.balustinum,"Bae, Y.M., Zheng, L., Hyun, J.E., Jung, K.S., ...",0.000844,43.037977,0.285151,1,0.008629,3.497856,27.046819
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4230,4230,295,0.057355,2.447187,Lactobaciulus plantarum,"Zwietering, M.H., De Wit, J.C., Cuppers, H.G.A...",12619.689989,26.384286,27151.735748,295,2.447187,9.314257,0.000000
4231,4231,295,2.492604,2.327517,Lactobaciulus plantarum,"Zwietering, M.H., De Wit, J.C., Cuppers, H.G.A...",12619.689989,26.384286,27151.735748,295,2.447187,9.314257,0.000000
4232,4232,295,1.743012,2.485061,Lactobaciulus plantarum,"Zwietering, M.H., De Wit, J.C., Cuppers, H.G.A...",12619.689989,26.384286,27151.735748,295,2.447187,9.314257,0.000000
4233,4233,295,0.994529,2.182619,Lactobaciulus plantarum,"Zwietering, M.H., De Wit, J.C., Cuppers, H.G.A...",12619.689989,26.384286,27151.735748,295,2.447187,9.314257,0.000000


In [4]:
e = np.exp(1) #set contant

In [5]:
#creat a dictionary to store all the data 
def All_data(id,df):
    all_data = {
        'ID'  : id,
        'xval': np.asarray(df.Time[df.ID == id]),
        'yval': np.asarray(df.PopBio[df.ID == id]),
        'N0'  : df.N0[df.ID == id].iloc[0],
        'Nmax': df.Nmax[df.ID == id].iloc[0],
        'Rmax': df.Rmax[df.ID == id].iloc[0],
        'Tlag': df.Tlag[df.ID == id].iloc[0],
        'Tmax': df.Tmax[df.ID == id].iloc[0],
        'A'   : df.A[df.ID == id].iloc[0],
        'H0'  : df.H0[df.ID == id].iloc[0]
    }
    return all_data

In [1]:
# Model_1 : classical model
def classical_residuals(params,x,data):
    N0 = params['N0'].value
    Nmax = params['Nmax'].value
    Rmax = params['Rmax'].value
    
    model_1 = N0*Nmax*(e**(Rmax*x))/(Nmax+N0*(e**(Rmax*x)-1))
    return model_1 - data

In [2]:
#creat a function to try to use lmfit to find the best fitted parameters in classical_model
def try_classical_residuals(id,df,max_tries):
    #use non-linear least square to fit for each curve in terms of classical model
    all_data = All_data(id,df) #get the used data from model fitting
    xval = all_data['xval']
    yval = all_data['yval']
    
    #creat a dinctionary to store results
    results = {'ID'    : all_data['ID'],
               'N0'    : all_data['N0'],
               'Nmax'  : all_data['Nmax'],
               'Rmax'  : all_data['Rmax'],
               'chisqr': [np.NaN],
               'RSS'   : [np.NaN],
               'TSS'   : [np.NaN],
               'Rsqrd' : [np.NaN],
               'Aic'   : [np.NaN],
               'Bic'   : [np.NaN]
              }
    
    trycount = 0
    while True: #use while loop, until the max tries is the last try,finding the best fitted parameters
        trycount += 1
        if results["Aic"]!= [np.NaN] or trycount > max_tries:
            break
        try:
            #use the estimsted startinf values when first try
            if trycount == 1:
                params = Parameters()
                params.add('N0', value = all_data["N0"], min = 0)
                params.add('Nmax', value = all_data["Nmax"], min = 0)
                params.add('Rmax', value = all_data["Rmax"], min = 0)
             # after the first try, the starting values will be selected from the range of [0,2*estimated_starting_values] randomly  
            else:
                params = Parameters()
                params.add('N0', value = np.random.uniform(0,all_data["N0"]*2, min = 0))
                params.add('Nmax', value = np.random.uniform(0,all_data["Nmax"]*2, min = 0))
                params.add('Rmax', value = np.random.uniform(0,all_data["Rmax"]*2, min = 0))
            #use minimize function to get the outcome    
            out = minimize(classical_residuals, params, args = (xval,yval))
            
            #calculate rsquared
            RSS = sum(classical_residuals(out.params,xval,yval)**2)
            TSS = sum((yval-np.mean(yval))**2)
            Rsquared = 1 - (RSS/TSS)
            
            #store results
            if results["Aic"] == [np.NaN]:
                results = {'ID'     : [id],
                           'N0'     : [out.params["N0"].value],
                           'Nmax'   : [out.params["Nmax"].value],
                           'Rmax'   : [out.params["Rmax"].value],
                           'chisqr' : [out.chisqr],
                           'RSS'    : RSS,
                           'TSS'    : TSS,
                           'Rsqrd'  : Rsquared,
                           'Aic'    : [out.aic],
                           'Bic'    : [out.bic]
                           }
            continue
        
        #if not converge then go to next try
        except ValueError:
            continue
            
        #convert results dictionary to a dataframe
        results = pd.DataFrame(results)
        return results
        

In [8]:
#Model_2 : Gompertz model
def gompertz_residuals(params,x,data):
    Tlag = params['Tlag'].value
    A    = params['A'].value
    Rmax = params['Rmax'].value
    
    model_2 = A*e**(-e**((Rmax*e*(Tlag-x))/A)+1)
    return model_2 - data

In [9]:
#creat a function to try to use lmfit to find the best fitted parameters in gompertz_model
def try_gompertz_residuals(id,df,max_tries):
    #use non-linear least square to fit for each curve in terms of gompertz model
    all_data = All_data(id,df) #get the used data from model fitting
    xval = all_data['xval']
    yval = all_data['yval']
    
    #creat a dinctionary to store results
    results = {'ID'    : all_data['ID'],
               'A'    : all_data['A'],
               'Tlag'  : all_data['Tlag'],
               'Rmax'  : all_data['Rmax'],
               'chisqr': [np.NaN],
               'RSS'   : [np.NaN],
               'TSS'   : [np.NaN],
               'Rsqrd' : [np.NaN],
               'Aic'   : [np.NaN],
               'Bic'   : [np.NaN]
              }
    
    trycount = 0
    while True: #use while loop, until the max tries is the last try,finding the best fitted parameters
        trycount += 1
        if results["Aic"]!= [np.NaN] or trycount > max_tries:
            break
        try:
            #use the estimsted startinf values when first try
            if trycount == 1:
                params = Parameters()
                params.add('A', value = all_data["A"], min = 0)
                params.add('Tlag', value = all_data["Tlag"], min = 0)
                params.add('Rmax', value = all_data["Rmax"], min = 0)
             # after the first try, the starting values will be selected from the range of [0,2*estimated_starting_values] randomly  
            else:
                params = Parameters()
                params.add('A', value = np.random.uniform(0,all_data["A"]*2, min = 0))
                params.add('Tlag', value = np.random.uniform(0,all_data["Tlag"]*2, min = 0))
                params.add('Rmax', value = np.random.uniform(0,all_data["Rmax"]*2, min = 0))
            #use minimize function to get the outcome    
            out = minimize(gompertz_residuals, params, args = (xval,yval))
            
            #calculate rsquared
            RSS = sum(gompertz_residuals(out.params,xval,yval)**2)
            TSS = sum((yval-np.mean(yval))**2)
            Rsquared = 1 - (RSS/TSS)
            
            #store results
            if results["Aic"] == [np.NaN]:
                results = {'ID'     : [id],
                           'A'      : [out.params["A"].value],
                           'Tlag'   : [out.params["Tlag"].value],
                           'Rmax'   : [out.params["Rmax"].value],
                           'chisqr' : [out.chisqr],
                           'RSS'    : RSS,
                           'TSS'    : TSS,
                           'Rsqrd'  : Rsquared,
                           'Aic'    : [out.aic],
                           'Bic'    : [out.bic]
                           }
            continue
        
        #if not converge then go to next try
        except ValueError:
            continue
            
        #convert results dictionary to a dataframe
        results = pd.DataFrame(results)
        return results
        

In [10]:
# Model_3 : Baranyi model
def baranyi_residuals(params,x,data):
    N0 = params['N0'].value
    Nmax = params['Nmax'].value
    Rmax = params['Rmax'].value
    H0   = params['H0'].value
    At = x+(1/Rmax)*(math.log((e**(-Rmax*x)+H0)/(1+H0)))
    model_3 = N0+Rmax*At-(math.log(1+((e**(Rmax*At)-1)/e**(Nmax-N0))))
    return model_3 - data

In [11]:
#creat a function to try to use lmfit to find the best fitted parameters in classical_model
def try_baranyi_residuals(id,df,max_tries):
    #use non-linear least square to fit for each curve in terms of classical model
    all_data = All_data(id,df) #get the used data from model fitting
    xval = all_data['xval']
    yval = all_data['yval']
    
    #creat a dinctionary to store results
    results = {'ID'    : all_data['ID'],
               'N0'    : all_data['N0'],
               'Nmax'  : all_data['Nmax'],
               'Rmax'  : all_data['Rmax'],
               'H0'    : all_data['H0'],
               'chisqr': [np.NaN],
               'RSS'   : [np.NaN],
               'TSS'   : [np.NaN],
               'Rsqrd' : [np.NaN],
               'Aic'   : [np.NaN],
               'Bic'   : [np.NaN]
              }
    
    trycount = 0
    while True: #use while loop, until the max tries is the last try,finding the best fitted parameters
        trycount += 1
        if results["Aic"]!= [np.NaN] or trycount > max_tries:
            break
        try:
            #use the estimsted startinf values when first try
            if trycount == 1:
                params = Parameters()
                params.add('N0', value = all_data["N0"], min = 0)
                params.add('Nmax', value = all_data["Nmax"], min = 0)
                params.add('Rmax', value = all_data["Rmax"], min = 0)
                params.add('H0', value = all_data["H0"], min = 0)
             # after the first try, the starting values will be selected from the range of [0,2*estimated_starting_values] randomly  
            else:
                params = Parameters()
                params.add('N0', value = np.random.uniform(0,all_data["N0"]*2, min = 0))
                params.add('Nmax', value = np.random.uniform(0,all_data["Nmax"]*2, min = 0))
                params.add('Rmax', value = np.random.uniform(0,all_data["Rmax"]*2, min = 0))
                params.add('H0', value = np.random.uniform(0,all_data["H0"]*2, min = 0))
            #use minimize function to get the outcome    
            out = minimize(baranyi_residuals, params, args = (xval,yval))
            
            #calculate rsquared
            RSS = sum(baranyi_residuals(out.params,xval,yval)**2)
            TSS = sum((yval-np.mean(yval))**2)
            Rsquared = 1 - (RSS/TSS)
            
            #store results
            if results["Aic"] == [np.NaN]:
                results = {'ID'     : [id],
                           'N0'     : [out.params["N0"].value],
                           'Nmax'   : [out.params["Nmax"].value],
                           'Rmax'   : [out.params["Rmax"].value],
                           'H0'     : [out.params["H0"].value],
                           'chisqr' : [out.chisqr],
                           'RSS'    : RSS,
                           'TSS'    : TSS,
                           'Rsqrd'  : Rsquared,
                           'Aic'    : [out.aic],
                           'Bic'    : [out.bic]
                           }
            continue
        
        #if not converge then go to next try
        except ValueError:
            continue
            
        #convert results dictionary to a dataframe
        results = pd.DataFrame(results)
        return results

In [12]:
#Model_4 : Buchanan model
def buchanan_residuals(params,x,data):
    N0 = params['N0'].value
    Nmax = params['Nmax'].value
    Rmax = params['Rmax'].value
    Tlag = params['Tlag'].value
    Tmax = params['Tmax'].value
    if x <= Tlag:
        model_4 = N0
    if x >= Tmax:
        model_4 = Nmax
    else:
        model_4 = Nmax+Rmax*(x-Tlag)
        
    return model_4 - data

In [13]:
#creat a function to try to use lmfit to find the best fitted parameters in Buchanan_model
def try_buchanan_residuals(id,df,max_tries):
    #use non-linear least square to fit for each curve in terms of Buchanan model
    all_data = All_data(id,df) #get the used data from model fitting
    xval = all_data['xval']
    yval = all_data['yval']
    
    #creat a dinctionary to store results
    results = {'ID'    : all_data['ID'],
               'N0'    : all_data['N0'],
               'Nmax'  : all_data['Nmax'],
               'Rmax'  : all_data['Rmax'],
               'Tmax'  : all_data['Tmax'],
               'Tlag'  : all_data['Tlag'],
               'chisqr': [np.NaN],
               'RSS'   : [np.NaN],
               'TSS'   : [np.NaN],
               'Rsqrd' : [np.NaN],
               'Aic'   : [np.NaN],
               'Bic'   : [np.NaN]
              }
    
    trycount = 0
    while True: #use while loop, until the max tries is the last try,finding the best fitted parameters
        trycount += 1
        if results["Aic"]!= [np.NaN] or trycount > max_tries:
            break
        try:
            #use the estimsted startinf values when first try
            if trycount == 1:
                params = Parameters()
                params.add('N0', value = all_data["N0"], min = 0)
                params.add('Nmax', value = all_data["Nmax"], min = 0)
                params.add('Rmax', value = all_data["Rmax"], min = 0)
                params.add('Tmax', value = all_data["Tmax"], min = 0)
                params.add('Tlag', value = all_data["Tlag"], min = 0)
             # after the first try, the starting values will be selected from the range of [0,2*estimated_starting_values] randomly  
            else:
                params = Parameters()
                params.add('N0', value = np.random.uniform(0,all_data["N0"]*2, min = 0))
                params.add('Nmax', value = np.random.uniform(0,all_data["Nmax"]*2, min = 0))
                params.add('Rmax', value = np.random.uniform(0,all_data["Rmax"]*2, min = 0))
                params.add('Tmax', value = np.random.uniform(0,all_data["Tmax"]*2, min = 0))
                params.add('Tlag', value = np.random.uniform(0,all_data["Tlag"]*2, min = 0))
            #use minimize function to get the outcome    
            out = minimize(buchanan_residuals, params, args = (xval,yval))
            
            #calculate rsquared
            RSS = sum(buchanan_residuals(out.params,xval,yval)**2)
            TSS = sum((yval-np.mean(yval))**2)
            Rsquared = 1 - (RSS/TSS)
            
            #store results
            if results["Aic"] == [np.NaN]:
                results = {'ID'     : [id],
                           'N0'     : [out.params["N0"].value],
                           'Nmax'   : [out.params["Nmax"].value],
                           'Rmax'   : [out.params["Rmax"].value],
                           'Tmax'   : [out.params["Rmax"].value],
                           'Tlag'   : [out.params["Rmax"].value],
                           'chisqr' : [out.chisqr],
                           'RSS'    : RSS,
                           'TSS'    : TSS,
                           'Rsqrd'  : Rsquared,
                           'Aic'    : [out.aic],
                           'Bic'    : [out.bic]
                           }
            continue
        
        #if not converge then go to next try
        except ValueError:
            continue
            
        #convert results dictionary to a dataframe
        results = pd.DataFrame(results)
        return results